In [1]:
import numpy as np
import pandas as pd

## Format temperature anomalies

### Create dataset temp-land-ocean-global.csv

In [38]:
path_temp_data = 'data/raw/temperature/'
path_global_land_ocean_file = 'global-land-ocean.txt'

In [39]:
data = pd.read_fwf(path_temp_data + path_global_land_ocean_file)
data.columns = ["Year", "Annual Anomaly", "Annual Unc.", "Five-year Anomaly", "Five-year Unc.", "Annual Anomaly", "Annual Unc.", "Five-year Anomaly", "Five-year Unc."]

ValueError: Length mismatch: Expected axis has 3 elements, new values have 9 elements

In [40]:
data.head(3)

,%,This file contains a brief summary of the changes in Earth's global average,Unnamed: 2
0,%,surface temperature estimated by combining the...,NaN
1,%,temperature field with a reinterpolated versio...,NaN
2,%,field.,NaN
